# 問題設定

## 基本設定

In [1]:
import numpy as np

In [2]:
n = 6
V = np.arange(n)
C = 9
B = {i:C for i in range(n)}
w = [3, 2, 5, 6, 1, 1]
E = [(0,2), (1,2)]
M = [(0,3), (1,4,5), (2,)]
r = [1]*len(M)
bin_cost = 100

## コンフリクト辞書とノットコンフリクト辞書の作成

### コンフリクトがあるノード間に枝があるような辞書 E_dict

In [3]:
def E_dict_func(E):
    E_dict={}
    flag=E[0][0]
    for i in range(len(E)):
        if flag!=E[i][0]:
            flag=E[i][0]
        E_dict[E[i][0]] = E_dict.setdefault(E[i][0],()) + (E[i][1],)
        E_dict[E[i][1]] = E_dict.setdefault(E[i][1],()) + (E[i][0],)
        
    for i in range(n):
        if i not in E_dict.keys():
            E_dict[i] = ()
            
    return E_dict

In [4]:
E_dict = E_dict_func(E)
E_dict

{0: (2,), 2: (0, 1), 1: (2,), 3: (), 4: (), 5: ()}

### コンフリクトがないノード間に枝があるような辞書 nonconf_E_dict

In [5]:
def nonconf_E_dict_func(E):
    import numpy as np
    N = np.arange(0,n) #辞書をつくる際にのみ使う定数
    E_dict = {}
    nonconf_E_dict={}
    flag=E[0][0]
    E_dict = E_dict_func(E)
    for i in range(len(E)):
        if flag!=E[i][0]:
            flag=E[i][0]
        nonconf_E_dict[E[i][0]] = tuple(set(N)-set(E_dict[E[i][0]])-{E[i][0]})
        nonconf_E_dict[E[i][1]] = tuple(set(N)-set(E_dict[E[i][1]])-{E[i][1]})
        
    for i in range(n):
        if len(E_dict[i])==0:
            nonconf_E_dict[i] = tuple(range(n))
    
    return nonconf_E_dict

In [6]:
nonconf_E_dict = nonconf_E_dict_func(E)
nonconf_E_dict

{0: (1, 3, 4, 5),
 2: (3, 4, 5),
 1: (0, 3, 4, 5),
 3: (0, 1, 2, 3, 4, 5),
 4: (0, 1, 2, 3, 4, 5),
 5: (0, 1, 2, 3, 4, 5)}

### コンフリクトがないノード間に枝があるとした場合の枝集合 nonconf_E

In [7]:
def nonconf_E_func(E):
    nonconf_E = []
    nonconf_E_dict = nonconf_E_dict_func(E)
    for i in nonconf_E_dict:
        for j in nonconf_E_dict[i]:
            nonconf_E.append((i,j))
            
    return nonconf_E

In [8]:
nonconf_E = nonconf_E_func(E)
nonconf_E

[(0, 1),
 (0, 3),
 (0, 4),
 (0, 5),
 (2, 3),
 (2, 4),
 (2, 5),
 (1, 0),
 (1, 3),
 (1, 4),
 (1, 5),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3),
 (3, 4),
 (3, 5),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3),
 (4, 4),
 (4, 5),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5)]

# アルゴリズム

In [9]:
def capacity(w_i,bin_b,w,C): #bin_bの容量がw_iの大きさよりも大きいかを判定
    if (C-sum(w[i] for i in bin_b)) >= w_i:
        return True
    else:
        return False

In [10]:
def conflict(item,bin_b,E_dict): #bin_bに割り当てられているアイテムたちとw_iの間にコンフリクトがあるかを判定
    if item not in E_dict:
        return False
    for i in bin_b:
        if i in E_dict[item]:
                return False
    return True

In [11]:
def group(i,G): #アイテムiの属するグループの番号を返す関数
    for index,menber in enumerate(G):
        if i in menber:
            return index

## グループを並べ替える関数
### 1.グループのメンバーが多い順

### 2.コンフリクトの数の平均が多い順

### 3.コンフリクトの数の最高値が高い順

### 4.グループを分けるコストが大きい順

In [12]:
#グループを並べ替える関数_1
def sort_G_1(G):
    return sorted(G,key=lambda x:len(x),reverse=True)

In [13]:
#グループを並べ替える関数_2
def sort_G_2(G):
    return sorted(G,key=lambda x:sum(len(E_dict[i]) for i in x)/len(x),reverse=True)

In [14]:
#グループを並べ替える関数_3
def sort_G_3(G):
    return sorted(G,key=lambda x:max(len(E_dict[i]) for i in x),reverse=True)

In [15]:
#グループを並べ替える関数_4
def sort_G_4(G):
    return sorted(enumerate(G),key=lambda x:r[x[0]],reverse=True)

# H1 : FFD

In [ ]:
def FFD(n,V,B,w,C,E,G):
    assign = {}
    penalty = {}
    I = sorted(V,key=lambda x:w[x],reverse=True) #大きさが大きい順にアイテムを並べ替える
    
    for i in I:
        group_no = group(i,G)
        for b in B:
            if capacity(w[i],B[b],w,C): #容量制約を満たす
                assign[b].append(i) #ビンbにアイテムiを入れる
                

In [22]:
def FFD(I,w,C,C_dict,E_dict,G,bin_dict,bins_group_using): #アイテムをビンに割り当てる
    #I = sorted(I,key=lambda x:len(nonconf_E_dict[x]),reverse=True) #コンフリクトのないアイテム数が多い順にアイテムを走査
    I = sorted(I,key=lambda x:w[x],reverse=True) #大きさが大きい順にアイテムを走査
    
    for i in I:
        group_no = group(i,G)
        for b in range(len(w)): #インデックスが小さい順にビンを走査
            bin_dict[b] = bin_dict.setdefault(b,[])
            C_dict[b] = C_dict.setdefault(b,C)
            if capacity(w[i],bin_dict[b],w,C): #容量制約を満たす
                bin_dict[b].append(i) #ビンbにアイテムiを入れる
                bins_group_using[group_no] = bins_group_using.setdefault(group_no,[]) + [b] #グループgroup_noの使用するビンのインデックスを１つ追加
                break  
                
    return bin_dict,bins_group_using

# H2 : FFD for conflict

In [256]:
def FFD_for_conflict(I,w,C,C_dict,E_dict,G,bin_dict,bins_group_using): #コンフリクトの数が多い順にアイテムをビンに割り当てる
    I = sorted(I,key=lambda x:len(E_dict[x]),reverse=True) #コンフリクトの数が多い順にアイテムを走査
    #I = sorted(I,key=lambda x:w[x],reverse=True) #大きさが大きい順にアイテムを走査
    
    for i in I:
        group_no = group(i,G)
        for b in range(len(w)): #インデックスが小さい順にビンを走査
            bin_dict[b] = bin_dict.setdefault(b,[])
            C_dict[b] = C_dict.setdefault(b,C)
            if capacity(w[i],bin_dict[b],w,C) and conflict(i,bin_dict[b],E_dict): #容量制約を満たす　and　コンフリクトがない
                bin_dict[b].append(i) #ビンbにアイテムiを入れる
                bins_group_using[group_no] = bins_group_using.setdefault(group_no,[]) + [b] #.append(b) #グループgroup_noの使用するビンのインデックスを１つ追加
                break  
                
    return bin_dict,bins_group_using

# H3 : FFD for conflict and group

In [257]:
def FFD_for_conflict_group(I,w,C,E_dict,G): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
    bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
    C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書
    
    G = sort_G_1(G) #グループの並べ替え
    for g in G: #グループごとにFFD_for_conflict()
        bin_dict,bins_group_using = FFD_for_conflict(g,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #gに対してFFD for conflictを行う
        for b in range(len(bin_dict)):
            C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
        
    return bin_dict,bins_group_using

# H4 : FFD for conflict and group 1

In [258]:
def H4(I,w,C,E_dict,G): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
    bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
    C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書
    
    H = [i for i in I if len(E_dict[i])==0] #アイテムの順序付き集合I内のコンフリクトの数が0のアイテムの集合をHとし、I=I\Hとする。
    G_prime = [tuple(set(g)-(set(H)&set(g))) for index,g in enumerate(G)] #グループの順序つき集合Gを更新する。
    
    G_prime = sort_G_1(G_prime) #グループの並べ替え
    for g in G_prime: #グループごとにFFD_for_conflict()
        bin_dict,bins_group_using = FFD_for_conflict(g,w,C,C_dict,E_dict,G_prime,bin_dict,bins_group_using) #gに対してFFD for conflictを行う
        for b in range(len(bin_dict)):
            C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
            
    bin_dict,bins_group_using = FFD(H,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #Hに対してFFDを行う
    for b in range(len(bin_dict)):
        C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b])
        
    return bin_dict,bins_group_using

# H5 : Improved FFD for conflict and group 2

In [259]:
def H5(I,w,C,E_dict,G): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
    bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
    C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書
    
    G = sort_G_1(G) #グループの並べ替え
    for g in G: #グループごとにFFD_for_conflict()
        H = [i for i in g if len(E_dict[i])==0] #コンフリクトの数が0のアイテムの集合H
        g = tuple(set(g)-set(H)) #g=g\H
        bin_dict,bins_group_using = FFD_for_conflict(g,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #gに対してFFD for conflictを行う
        bin_dict,bins_group_using = FFD(H,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #Hに対してFFDを行う
        for b in range(len(bin_dict)):
            C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
        
    return bin_dict,bins_group_using

# H6 : Improved FFD for conflict and group 3

In [260]:
def H6(I,w,C,E_dict,G): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
    bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
    C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書
    
    G = sort_G_3(G) #グループの並べ替え
    for g in G:
        H = [i for i in g if len(E_dict[i])==0] #コンフリクトの数が0のアイテムの集合H
        g = tuple(set(g)-set(H)) #g=g\H
        while len(g)>0:
            max_degree = max(i for i in g) #グループg内で最もコンフリクトの数が多いアイテムiを選ぶ
            g_i = (max_degree,) + tuple(menber for menber in g if menber in nonconf_E_dict[max_degree]) #グループg内でiとコンフリクトがないアイテムとiからなる集合g_iを定める
            g = tuple(set(g)-set(g_i)) #g=g\g_i
            bin_dict,bins_group_using = FFD_for_conflict(g_i,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #g_iに対してFFD for conflictを行う
            for b in range(len(bin_dict)):
                C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
        bin_dict,bins_group_using = FFD(H,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #Hに対してFFDを行う
        for b in range(len(bin_dict)):
            C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく

    return bin_dict,bins_group_using

# H7 : Improved FFD for conflict and group 4

In [261]:
def H7(I,w,C,E_dict,G): #コンフリクトの数が多い順にアイテムをビンに割り当てる、をグループごとに行う
    bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
    bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
    C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書
    
    H = [i for i in I if len(E_dict[i])==0] #アイテムの順序付き集合I内のコンフリクトの数が0のアイテムの集合をHとし、I=I\Hとする。
    G_prime = [tuple(set(g)-(set(H)&set(g))) for index,g in enumerate(G)] #グループの順序つき集合Gを更新する。
    
    G_prime = sort_G_3(G_prime) #グループの並べ替え
    for g in G_prime:
        g = tuple(set(g)-set(H)) #g=g\H
        while len(g)>0:
            max_degree = max(i for i in g) #グループg内で最もコンフリクトの数が多いアイテムiを選ぶ
            g_i = (max_degree,) + tuple(menber for menber in g if menber in nonconf_E_dict[max_degree]) #グループg内でiとコンフリクトがないアイテムとiからなる集合g_iを定める
            g = tuple(set(g)-set(g_i)) #g=g\g_i
            bin_dict,bins_group_using = FFD_for_conflict(g_i,w,C,C_dict,E_dict,G_prime,bin_dict,bins_group_using) #g_iに対してFFD for conflictを行う
            for b in range(len(bin_dict)):
                C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
    bin_dict,bins_group_using = FFD(H,w,C,C_dict,E_dict,G,bin_dict,bins_group_using) #Hに対してFFDを行う
    for b in range(len(bin_dict)):
        C_dict[b] = C_dict[b]-sum(w[i] for i in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく

    return bin_dict,bins_group_using

# Gendreauさんのビンパッキングに対するヒューリスティクスなやつ

1. アイテムの集合V内のコンフリクトの数が0のアイテムの集合をHとし、V=V \Hとする。
2. Johnsonのヒューリスティクスを使ってコンフリクトグラフG内のクリークDを定め、V=V \Dとする。V=φならば終了。
3. クリークDのすべての要素iに対し、Johnsonのヒューリスティックを使ってV ∪ {i}として導出されるノンコンフリクトグラフG^~iの中からiを含む最大のクリークDiを定める。Diに対しFFDを行う。Vからiを含むビンに割り当てられているアイテム全てを除く。そして、D=D\iとする。もしI=φならばステップ4へ。もしD=φならばステップ2へ。さもなければ、ステップ3へ。
4. Hに対してFFDを行う。このとき、現在使われているビンから優先的に割り当てるようにする。

# Johnsonさんの最大クリークを求めるヒューリスティクスなやつ

入力　G=(N,A)  
出力 最大のクリーク
1. SUB =Nとする
2. もしSUBがクリークなら終了し、SUBを返す
3. yをSUBに属するノードのうち、SUBに属する他のノードと隣接している数が最も少ないものとする
4. SUB=SUB-{y}とし、2.へ

In [16]:
def clique(SUB,A):
    flag=True
    for i in SUB:
        for j in SUB:
            if i!=j:
                if (i,j) not in A and (j,i) not in A:
                    flag = False
                    break
        if flag!=True:
            break       
    return flag

In [17]:
def fewest_degree_node(SUB,A):
    degree_dict = {}
    for n in SUB:
        degree_dict[n] = 0
    for e in A:
        try:
            degree_dict[e[0]] += 1
            degree_dict[e[1]] += 1
        except:
            continue
    return {min(degree_dict,key=lambda x:degree_dict[x])}

In [18]:
def Johnson(N,A): #与えられた点集合と枝集合から最大クリークを生成する #Nは点の集合、Aは枝の集合
    SUB = N
    while True:
        if clique(SUB,A):
            break
        else:
            y = fewest_degree_node(SUB,A) #SUBのノードのうち、最も次数の少ないものを定める
            SUB = SUB-y #SUBからyを除く
            for e in A:
                if y in e:
                    A = A-{e} #yにつながってる枝を除く
    return SUB

In [20]:
I = sorted(V,key=lambda x:w[x],reverse=True)

In [24]:
G = [(0,3), (1,4,5), (2,)]

In [25]:
bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書

#1
V = set(I)
H = set(i for i in V if len(E_dict[i])==0) #アイテムの集合V内のコンフリクトの数が0のアイテムの集合をHとし、V=V\Hとする。
V = V-H #コンフリクトの数が0のアイテムを含まないアイテム集合
E = set(E)    
#print("V \t",V)

#2
if V==set(): #V=φならば終了
    pass
else:
    D = Johnson(V,E) #DはグラフG内の最大のクリーク
    V = V-D
#print("D \t",D)
#print("V-D")
#print("V \t",V)
#print()

#3
for i in D:
    print("\t i \t",i)
    
    V = V | {i}
    print("\t V \t",V)
    
    D_i = Johnson(V,nonconf_E) #V ∪ {i} と　コンフリクトがないアイテム同士の組みを要素にもつ枝集合　から最大クリークを見つける
    print("\t D_i \t",D_i)
    
    bin_dict,bins_group_using = FFD(D_i,w,C,C_dict,E_dict,G,bin_dict,bins_group_using)
    for index,value in bin_dict.items():
        if i in value:
            V = V - set(value)
        else:
            bin_dict[index] = []
            
    print("\t bin_dict \t",bin_dict)
    print("\t bins_group_using \t",bins_group_using)
    
    for b in range(len(bin_dict)):
        C_dict[b] = C_dict[b]-sum(w[k] for k in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
        
    D = D - {i}
    print("\t V \t",V)
    print("\t D \t",D)
    print("\t C_dict \t",C_dict)
    print()
    
    #4
    #if V==set() and D==set():

	 i 	 1
	 V 	 {0, 1}
	 D_i 	 {0, 1}
	 bin_dict 	 {0: [0, 1]}
	 bins_group_using 	 {0: [0], 1: [0]}
	 V 	 set()
	 D 	 {2}
	 C_dict 	 {0: 4}

	 i 	 2
	 V 	 {2}
	 D_i 	 {2}
	 bin_dict 	 {0: [], 1: [2]}
	 bins_group_using 	 {0: [0], 1: [0], 2: [1]}
	 V 	 set()
	 D 	 set()
	 C_dict 	 {0: 4, 1: 4}



In [26]:
bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書

#1
V = set(I)
H = set(i for i in V if len(E_dict[i])==0) #アイテムの集合V内のコンフリクトの数が0のアイテムの集合をHとし、V=V\Hとする。
V = V-H #コンフリクトの数が0のアイテムを含まないアイテム集合
E = set(E)    
print("V \t",V)
#2
if V==set(): #V=φならば終了
    pass
else:
    D = Johnson(V,E) #DはグラフG内の最大のクリーク
    V = V-D
print("D \t",D)
print("V-D")
print("V \t",V)
print()
#3
bin_dict_temp = {}
bins_group_using_temp = {}
for i in D:
    print("\t i \t",i)
    
    V = V | {i}
    print("\t V \t",V)
    
    D_i = Johnson(V,nonconf_E) #V ∪ {i} と　コンフリクトがないアイテム同士の組みを要素にもつ枝集合　から最大クリークを見つける
    print("\t D_i \t",D_i)
    
    bin_dict_temp,bins_group_using_temp = FFD(D_i,w,C,C_dict,E_dict,G,bin_dict_temp,bins_group_using_temp)
    print("\t bin_dict_temp \t",bin_dict_temp)
    print("\t bins_group_using_temp \t",bins_group_using_temp)
    
    for index,value in bin_dict_temp.items():
        if i in value:
            V = V - set(value)
            bin_dict[index] = value
        """for j in value:
            group_no = group(j,G)
            bins_group_using[group_no] = bins_group_using.setdefault(group_no,[]) + [index]"""
        
    for b in range(len(bin_dict)):
        if bin_dict[b]==9:
            C_dict[b] = C-sum(w[k] for k in bin_dict[b]) #割り当てたアイテムの大きさ分ずつビンの容量を減らしていく
        
    bins_group_using = bins_group_using_temp
    #bin_dict_temp,bins_group_using_temp = bin_dict,bins_group_using
    
    print("\t bin_dict \t",bin_dict)
    print("\t bins_group_using \t",bins_group_using)
    D = D - {i}
    print("\t V \t",V)
    print("\t D \t",D)
    print("\t C_dict \t",C_dict)
    print()
    
    #4
    #if V==set() and D==set():

V 	 {0, 1, 2}
D 	 {1, 2}
V-D
V 	 {0}

	 i 	 1
	 V 	 {0, 1}
	 D_i 	 {0, 1}
	 bin_dict_temp 	 {0: [0, 1]}
	 bins_group_using_temp 	 {0: [0], 1: [0]}
	 bin_dict 	 {0: [0, 1]}
	 bins_group_using 	 {0: [0], 1: [0]}
	 V 	 set()
	 D 	 {2}
	 C_dict 	 {0: 9}

	 i 	 2
	 V 	 {2}
	 D_i 	 {2}
	 bin_dict_temp 	 {0: [0, 1], 1: [2]}
	 bins_group_using_temp 	 {0: [0], 1: [0], 2: [1]}
	 bin_dict 	 {0: [0, 1], 1: [2]}
	 bins_group_using 	 {0: [0], 1: [0], 2: [1]}
	 V 	 set()
	 D 	 set()
	 C_dict 	 {0: 9, 1: 9}



In [139]:
def Gendreau(I,w,C,E,nonconf_E,E_dict,G):
    bin_dict = {} #ビンのインデックスをキーとし、ビンに割り当てられたアイテムを値とする辞書
    bins_group_using = {} #グループのインデックスをキーとし、グループが使用したビンのインデックスのリストを値とする辞書
    C_dict = {} #ビンのインデックスをキーとし、ビンの残り容量を値とする辞書
    
    #1
    V = set(I)
    H = set(i for i in V if len(E_dict[i])==0) #アイテムの集合V内のコンフリクトの数が0のアイテムの集合をHとし、V=V\Hとする。
    V = V-H #コンフリクトの数が0のアイテムを含まないアイテム集合
    E = set(E)    
    #2
    if V==set(): #V=φならば終了
        pass
    else:
        D = Johnson(V,E) #DはグラフG内の最大のクリーク
        V = V-D
    #3
    for i in D:
        V = V | {i}
        D_i = Johnson(V,nonconf_E) #V ∪ {i} と　コンフリクトがないアイテム同士の組みを要素にもつ枝集合　から最大クリークを見つける
        bin_dict_temp = {}
        bins_group_using_temp = {}
        bin_dict_temp,bins_group_using_temp = FFD(I,w,C,C_dict,E_dict,G,bin_dict,bins_group_using)
        for index,value in enumerate(bins_group_using_temp):
            if i in value:
                

In [91]:
Gendreau(I,w,C,E,nonconf_E,G)

{(1, 2), (0, 2)}


NameError: name 'Johnson' is not defined

# H3 : FFD for conflict and groupの実行

In [19]:
I = list(range(n))
C = maximum #{b:maximum+1 for b in range(n)}

In [20]:
bin_dict,bins_group_using = FFD_for_conflict_group(I,w,C,E_dict,G)

In [21]:
item_number = 0
for i in range(len(bin_dict)):
    item_number += len(bin_dict[i])
    print(i,bin_dict[i])
    
print(item_number)

0 [1, 4, 5, 0]
1 [3]
2 [2]
6


In [22]:
print(bins_group_using)

{0: [0, 0, 0], 1: [0, 1], 2: [2]}


In [23]:
for group_number in range(len(G)):
    print(group_number,bins_group_using[group_number])
    #print(r[group_number]*len(set(bins_group_using[group_number])))

0 [0, 0, 0]
1 [0, 1]
2 [2]


In [24]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

3


# H4 : Improved FFD for conflict and group 1の実行

In [25]:
I = list(range(n))
C = maximum #{b:maximum+1 for b in range(n)}

In [26]:
bin_dict,bins_group_using = H4(I,w,C,E_dict,G)

In [27]:
item_number = 0
for i in range(len(bin_dict)):
    item_number += len(bin_dict[i])
    print(i,bin_dict[i])
    
print(item_number)

0 [0, 1, 4, 5]
1 [2]
2 [3]
6


In [28]:
print(bins_group_using)

{0: [0, 2], 1: [0, 0, 0], 2: [1]}


In [29]:
for group_number in range(len(G)):
    print(group_number,bins_group_using[group_number])
    #print(r[group_number]*len(set(bins_group_using[group_number])))

0 [0, 2]
1 [0, 0, 0]
2 [1]


In [30]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

3


# H6 : Improved FFD for conflict and group 3の実行

In [31]:
I = list(range(n))
C = maximum #{b:maximum+1 for b in range(n)}

In [32]:
bin_dict,bins_group_using = H6(I,w,C,E_dict,G)

In [33]:
number = 0
for i in range(len(bin_dict)):
    number += len(bin_dict[i])
    print(i,bin_dict[i])
    
print(number)

0 [2, 4, 5]
1 [0, 3]
2 [1]
6


In [34]:
print(bins_group_using)

{0: [0], 1: [1, 1], 2: [2, 0, 0]}


In [35]:
for group_number in range(len(G)):
    print(r[group_number]*len(set(bins_group_using[group_number])))

1
1
2


In [36]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

3


# H7 : Improved FFD for conflict and group 4の実行

In [37]:
I = list(range(n))
C = maximum #{b:maximum+1 for b in range(n)}

In [38]:
bin_dict,bins_group_using = H7(I,w,C,E_dict,G)

In [39]:
number = 0
for i in range(len(bin_dict)):
    number += len(bin_dict[i])
    print(i,bin_dict[i])
    
print(number)

0 [2, 4, 5]
1 [0, 1]
2 [3]
6


In [40]:
print(bins_group_using)

{0: [0, 2], 1: [1, 0, 0], 2: [1]}


In [41]:
for group_number in range(len(G)):
    print(r[group_number]*len(set(bins_group_using[group_number])))

2
2
1


In [42]:
bin_counter=0
for i in range(len(bin_dict)):
    if len(bin_dict[i]) > 0:
        bin_counter+=1
print(bin_counter)

3


# グループの並べ替え、新しい方法を考える

# ビンを使うコストとグループを分けるコストの比によってH6 : FFD for conflict and group 3を使うアイテムの集合とGendreauのH6を使うアイテムの集合に分割する --> GendreauのH6実装

# H4とH7の完成 **H4は完成